In [1]:
class GaussInt:
    def __init__(self, real, imaginary=None):
        # allow native complex type construction or parameter construction
        if (imaginary == None):
            self.r = int(real.real)
            self.i = int(real.imag)
        else:
            self.r = real
            self.i = imaginary
        
    def __mul__(self, other):
        # construct GaussInt from non-GaussInt
        if not isinstance(other, GaussInt):
            other = GaussInt(other)      

        # (a + bi) (c + di) = ac - bd
        a, b, c, d = self.r, self.i, other.r, other.i
        r = (a * c) - (b * d)
        i = (a * d) + (b * c)
        return GaussInt(r, i)
    
    def divide(self, other):
        if not isinstance(other, GaussInt):
            r_r = int(round(self.r / other))
            r_i = int(round(self.i / other))
            return GaussInt(r_r, r_i), self - (GaussInt(r_r, r_i) * other)
        else:
            a, b, c, d = self.r, self.i, other.r, other.i
            conj = GaussInt(other.r, -other.i)
            # (a + bi) / (c + di)
            # (a + bi) * (c - di) / ((c + di) * (c - di))
            # (a + bi) * (c + di) / (c ** 2 - d ** 2)
            res = (self * conj) / (other * conj).r
            
            r_r = int(round(res.r))
            r_i = int(round(res.i))
            return GaussInt(r_r, r_i), self - (GaussInt(r_r, r_i) * other)
    
    def __truediv__(self, other):
        return self.divide(other)[0]
    
    def __mod__(self, other):
        return self.divide(other)[1]
        
    def __sub__(self, other):
        return self + (other * -1)
    
    def __add__(self, other):
        if not isinstance(other, GaussInt):
            other = GaussInt(other)
            
        return GaussInt(self.r + other.r, self.i + other.i)

    def __repr__(self):
        return "({}{}{}i)".format(self.r, "+" if (self.i >= 0) else "", self.i)
    
    def __eq__(self, other):
        # construct GaussInt from non-GaussInt
        if not isinstance(other, GaussInt):
            other = GaussInt(other)
        return (self.r == other.r) and (self.i == other.i)
    
    def __neq__(self, other):
        return not (self == other)
    
    def norm(self):
        return self.r ** 2 + self.i ** 2

In [2]:
# Recursive form of gcd
def gcd(a, b):
    return b if a == 0 else gcd(b%a, a)

In [3]:
# List comprension to find number of coprimes less than n
def naiive_tot(n):
    return len([i for i in range(n) if gcd(n, i) == 1])

# totient of a power of a prime
def p_tot(p, n):
    return (p - 1) * p ** (n - 1)

# finds the totient of a number using its factorialization
def smart_tot(n):
    out     = 1
    factors = fact(n)
    for i in factors:
        out *= p_tot(i, factors[i])
    return out

In [4]:
## Extended Euclidean Algorithm
def ext_euclid(a, b):
    a, b = sorted((a, b))
    
    # remainders
    r = [b, a]
    
    # coefficient of b
    s = [1, 0]
    
    # coefficient of a
    t = [0, 1]
    
    # compute values until remainder is 0
    i = 1
    while(r[i] != 0):
        q = (r[i - 1] // r[i])
        r.append(r[i - 1] - q * r[i])
        s.append(s[i - 1] - q * s[i])
        t.append(t[i - 1] - q * t[i])
        i += 1
        
    # return relevant coefficients and remainder
    return t[i - 1], s[i - 1], r[i - 1]

In [5]:
# Short prime finder
prime_list = [2] + [*filter(lambda i:all(i%j for j in range(3,i,2)), range(3,10000,2))]
prime_set = set(prime_list)

In [6]:
# Prime Factorialization
from collections import defaultdict

# Prime Factoring Algorithm
def fact(n):
    # dictionary with default value
    out = defaultdict(int)
    
    # fresh new prime list
    primes = prime_list.copy()

    f = primes.pop(0)
    while f <= n:
        if n % f == 0:
            out[f] += 1
            n //= f
        else:
            f = primes.pop(0)
    return out # [j for k in [([i] * out[i]) for i in out] for j in k]

In [7]:
# cathode ray tu-- sorry... Chinese Remainder Theorem
# x = a_k mod n_k
def crt(a: list, n: list):
    # first, verify lists are of same size
    assert len(a) == len(n)
    
    # next, verify coprimality and generate product
    N = 1
    for i in n:
        assert (gcd(i, N) == 1)
        N *= i
        
    # now, add element N / n_i = y_i to each n
    n = [(i, N // i) for i in n]
    
    # next, add element multiplicative inverse of y_i = z_i to each n
    n = [i + tuple([ext_euclid(*i)[0]]) for i in n]

    # now, return x = sum(a * y * z) and uniqueness factor
    return sum([a[i] * n[i][1] * n[i][2] for i in range(len(n))]) % N, N

In [8]:
# determines if a is a quadratic residue of p
def legendre(a, p, d = 2):
    # if p is prime, simply return the 
    if is_prime(p):
        symbol = ((a) ** ((p - 1) // d)) % p
        # handles negative -1
        return symbol if symbol < 2 else -1
    
    # otherwise, return product of prime factors
    product = 1
    factors = fact(p)
    for i in factors:
        product *= (legendre(i) ** factors[i])
    return product

In [9]:
def is_prime(n):
      return n in prime_set